In [1]:
using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra
using InvertedIndices
using MachineLearningPotential

# Symmetry Functions

Time to write and test the basics of SymmFunctions for ML potentials

In [2]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
rad_function = RadialType2{Float64}(0.001,11.338,[1.,1.])
ang_func = AngularType3{Float64}(0.0001,1.0,1.0,11.338,[1.,1.,1.])
ang_func2 = AngularType3{Float64}(0.0001,-1.0,2.0,11.338,[1.,1.,1.])

AngularType3{Float64}(0.0001, -1.0, 2.0, 11.338, [1, 1, 1])

Demonstrates the difference between [:,index] and [index,:]

# Angular Symmetry Function

Testing prior to type system

In [3]:
@benchmark calc_symmetry_function(positions,dis2mat,1,rad_function)

BenchmarkTools.Trial: 10000 samples with 186 evaluations.
 Range (min … max):  541.177 ns …  19.166 μs  ┊ GC (min … max): 0.00% … 95.96%
 Time  (median):     644.059 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   712.187 ns ± 817.921 ns  ┊ GC (mean ± σ):  5.43% ±  4.58%

  ▁▃▄▆▇█▇▇▅▄▃▁  ▁▂▂▂▃▂▁▁   ▁▂▂▄▄▅▆▆▆▆▆▅▄▃▂▂▁▂▁▂▁▁▁▁▁      ▁     ▃
  ███████████████████████▇███████████████████████████▇███████▆▆ █
  541 ns        Histogram: log(frequency) by time        935 ns <

 Memory estimate: 1.06 KiB, allocs estimate: 4.

In [4]:
@benchmark calc_symmetry_function(positions,dis2mat,1,ang_func)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  35.496 μs …  3.380 ms  ┊ GC (min … max): 0.00% … 98.83%
 Time  (median):     39.753 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   40.777 μs ± 41.873 μs  ┊ GC (mean ± σ):  1.43% ±  1.39%

       ▆ ▄▁ ▂▂██▇▅▂                                            
  ▂▂▄▄▅█▇██▇████████▆▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  35.5 μs         Histogram: frequency by time        54.7 μs <

 Memory estimate: 22.02 KiB, allocs estimate: 9.

In [5]:
calc_symmetry_function(positions,dis2mat,1,ang_func2)

5.982413095508253

With this all functioning we can now generate a vector __G__ of all the necessary symmetry functions, with zero for the Cu-Zn ones.

In [21]:
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]
radsymmvec = []
for row in eachrow(X)
    symmfunc = RadialType2{Float64}(row[3],row[5],[row[1],row[2]])
    push!(radsymmvec,symmfunc)
end
G = calc_symmetry_function.(Ref(positions),Ref(dis2mat),Ref(1),radsymmvec)

10×5 Matrix{Float64}:
 1.0  1.0  0.001  0.0  11.338
 1.0  0.0  0.001  0.0  11.338
 1.0  1.0  0.02   0.0  11.338
 1.0  0.0  0.02   0.0  11.338
 1.0  1.0  0.035  0.0  11.338
 1.0  0.0  0.035  0.0  11.338
 1.0  1.0  0.1    0.0  11.338
 1.0  0.0  0.1    0.0  11.338
 1.0  1.0  0.4    0.0  11.338
 1.0  0.0  0.4    0.0  11.338

In [25]:
V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.228],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]
T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]
angularsymmvec = []
for element in V 
    for types in T
        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types)
        push!(angularsymmvec,symmfunc)
    end
end

total_symm_vec = vcat(radsymmvec,angularsymmvec)

88-element Vector{Any}:
 RadialType2{Float64}(0.001, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.001, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.02, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.02, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.035, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.035, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.1, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.1, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.4, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.4, 11.338, [1.0, 0.0])
 ⋮
 AngularType3{Float64}(0.08, -1.0, 2.0, 11.338, [1.0, 1.0, 1.0])
 AngularType3{Float64}(0.08, -1.0, 2.0, 11.338, [1.0, 1.0, 0.0])
 AngularType3{Float64}(0.08, -1.0, 2.0, 11.338, [1.0, 0.0, 0.0])
 AngularType3{Float64}(0.08, -1.0, 4.0, 11.338, [1.0, 1.0, 1.0])
 AngularType3{Float64}(0.08, -1.0, 4.0, 11.338, [1.0, 1.0, 0.0])
 AngularType3{Float64}(0.08, -1.0, 4.0, 11.338, [1.0, 0.0, 0.0])
 AngularType3{Float64}(0.08, 1.0, 4.0, 11.338, [1.0, 1.0, 1.0])
 AngularType3{Float64}(0.08, 1.0, 4.0, 

In [36]:
#G = calc_symmetry_function.(Ref(positions),Ref(dis2mat),Ref(1),total_symm_vec)
function calc_symmetry_functions(positions,dis2mat,index,total_symm_vec)
   G = calc_symmetry_function.(Ref(positions),Ref(dis2mat),Ref(index),total_symm_vec)
   return G
end

calc_symmetry_functions (generic function with 2 methods)

In [37]:
@benchmark calc_symmetry_functions($positions,$dis2mat,$rand(1:55),$total_symm_vec)

BenchmarkTools.Trial: 5991 samples with 1 evaluation.
 Range (min … max):  644.773 μs …   4.091 ms  ┊ GC (min … max): 0.00% … 63.08%
 Time  (median):     800.098 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   833.430 μs ± 191.777 μs  ┊ GC (mean ± σ):  1.47% ±  5.34%

      ▁             █▆▂▂                                         
  ▂▂▄▅█▅▅▃▃▂▂▂▂▂▃▃▆▇████▇▅▅▄▃▃▃▃▃▂▂▂▃▃▃▃▄▆▅▄▃▄▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁ ▃
  645 μs           Histogram: frequency by time          1.1 ms <

 Memory estimate: 584.27 KiB, allocs estimate: 380.

In [31]:
scaledata = [1    1        0.450564942      15.523345849       9.298968199
   1    2        0.000000000      10.174902869       1.438899770
   1    3        0.100819173       7.607500154       4.564166627
   1    4        0.000000000       5.370905097       0.709810315
   1    5        0.032030703       4.808672275       2.842081607
   1    6        0.000000000       3.500046942       0.442888382
   1    7        0.000282276       1.068088052       0.518435876
   1    8        0.000000000       0.723935150       0.080510621
   1    9        0.000000000       0.008816804       0.000504980
   1   10        0.000000000       0.005508918       0.000082833
   1   11        0.019761299      40.773943091      16.211040424
   1   12        0.000000000      16.964164093       3.744682565
   1   13        0.000000000      17.681976100       0.630049442
   1   14        0.000298798       6.702709001       2.432733865
   1   15        0.000000000       3.416558145       0.575011527
   1   16        0.000000000       3.612414431       0.111869881
   1   17        0.001119279      11.348120314       4.318878188
   1   18        0.000000000       5.371376879       1.013577855
   1   19        0.000000000       5.908457969       0.188441258
   1   20        0.000150913       4.878271436       1.744170918
   1   21        0.000000000       2.554335099       0.412016771
   1   22        0.000000000       2.632922947       0.079981481
   1   23        0.000364749       6.849544568       2.557278109
   1   24        0.000000000       3.376636349       0.600469545
   1   25        0.000000000       3.607340627       0.110124966
   1   26        0.000046825       2.878786940       1.002024127
   1   27        0.000000000       1.573280329       0.236379102
   1   28        0.000000000       1.549897102       0.045670938
   1   29        0.003303832      14.200032632       5.513995416
   1   30        0.000000000       6.079408958       1.276308770
   1   31        0.000000000       6.178264992       0.203162950
   1   32        0.000816090       9.462506627       3.580829495
   1   33        0.000000000       4.381367884       0.832942042
   1   34        0.000000000       4.394339232       0.132458384
   1   35        0.000009239       1.435395759       0.478597502
   1   36        0.000000000       0.823661273       0.112636004
   1   37        0.000000000       0.760317722       0.021568547
   1   38        0.000000299       0.522720116       0.142634346
   1   39        0.000000000       0.309832127       0.033912440
   1   40        0.000000000       0.247575657       0.006668426
   1   41        0.000000000       0.090079385       0.016825229
   1   42        0.000000000       0.067772519       0.003835361
   1   43        0.000000000       0.046858786       0.000900378
   1   44        0.000008636       1.489259864       0.510144983
   1   45        0.000000000       0.807465898       0.119579453
   1   46        0.000000000       0.775663890       0.020777351
   1   47        0.000098860       4.035065427       1.447333466
   1   48        0.000000000       2.043154521       0.337146607
   1   49        0.000000000       1.896211614       0.051856333
   1   50        0.000091162       3.111755187       1.114760290
   1   51        0.000000000       1.537135560       0.259209885
   1   52        0.000000000       1.410305086       0.038908445
   1   53        0.000000027       0.194292865       0.045843998
   1   54        0.000000000       0.118433712       0.010830701
   1   55        0.000000000       0.080832326       0.002142876
   1   56        0.000000000       0.029633261       0.004300851
   1   57        0.000000000       0.021227057       0.000989535
   1   58        0.000000000       0.017849188       0.000230949
   1   59        0.000018308       0.364425483       0.106768740
   1   60        0.000000000       0.159351076       0.024046248
   1   61        0.000000000       0.104539311       0.003308901
   1   62        0.000001524       0.868360269       0.274262961
   1   63        0.000000000       0.518086542       0.063787099
   1   64        0.000000000       0.398922421       0.009272786
   1   65        0.000000011       0.118377361       0.029123321
   1   66        0.000000000       0.070159835       0.006789985
   1   67        0.000000000       0.053143804       0.001209256
   1   68        0.000000000       0.029992692       0.005394701
   1   69        0.000000000       0.019314966       0.001261105
   1   70        0.000000000       0.012884418       0.000252201
   1   71        0.000001229       0.403860766       0.126159974
   1   72        0.000000000       0.232066020       0.029222441
   1   73        0.000000000       0.169061121       0.004042407
   1   74        0.000000476       0.049633542       0.012792920
   1   75        0.000000000       0.026386759       0.002898889
   1   76        0.000000000       0.012642249       0.000383883
   1   77        0.000000001       0.091764438       0.018943889
   1   78        0.000000000       0.059877724       0.004369334
   1   79        0.000000000       0.035476065       0.000590275
   1   80        0.000000000       0.011450928       0.001749395
   1   81        0.000000000       0.006596024       0.000405090
   1   82        0.000000000       0.005160759       0.000064330
   1   83        0.000000000       0.002034305       0.000193585
   1   84        0.000000000       0.001159051       0.000044987
   1   85        0.000000000       0.000808883       0.000008699
   1   86        0.000000001       0.039557348       0.008284551
   1   87        0.000000000       0.026092508       0.001908692
   1   88        0.000000000       0.013706197       0.000248299]

88×5 Matrix{Float64}:
 1.0   1.0  0.450565     15.5233       9.29897
 1.0   2.0  0.0          10.1749       1.4389
 1.0   3.0  0.100819      7.6075       4.56417
 1.0   4.0  0.0           5.37091      0.70981
 1.0   5.0  0.0320307     4.80867      2.84208
 1.0   6.0  0.0           3.50005      0.442888
 1.0   7.0  0.000282276   1.06809      0.518436
 1.0   8.0  0.0           0.723935     0.0805106
 1.0   9.0  0.0           0.0088168    0.00050498
 1.0  10.0  0.0           0.00550892   8.2833e-5
 ⋮                                     
 1.0  80.0  0.0           0.0114509    0.00174939
 1.0  81.0  0.0           0.00659602   0.00040509
 1.0  82.0  0.0           0.00516076   6.433e-5
 1.0  83.0  0.0           0.00203431   0.000193585
 1.0  84.0  0.0           0.00115905   4.4987e-5
 1.0  85.0  0.0           0.000808883  8.699e-6
 1.0  86.0  1.0e-9        0.0395573    0.00828455
 1.0  87.0  0.0           0.0260925    0.00190869
 1.0  88.0  0.0           0.0137062    0.000248299

In [32]:
scales = []
for row in eachrow(scaledata)
    g_min,g_max = row[3],row[4]
    push!(scales,(g_min,g_max))
end
scales

88-element Vector{Any}:
 (0.450564942, 15.523345849)
 (0.0, 10.174902869)
 (0.100819173, 7.607500154)
 (0.0, 5.370905097)
 (0.032030703, 4.808672275)
 (0.0, 3.500046942)
 (0.000282276, 1.068088052)
 (0.0, 0.72393515)
 (0.0, 0.008816804)
 (0.0, 0.005508918)
 ⋮
 (0.0, 0.011450928)
 (0.0, 0.006596024)
 (0.0, 0.005160759)
 (0.0, 0.002034305)
 (0.0, 0.001159051)
 (0.0, 0.000808883)
 (1.0e-9, 0.039557348)
 (0.0, 0.026092508)
 (0.0, 0.013706197)

In [35]:
#@benchmark calc_symmetry_functions($positions,$dis2mat,$rand(1:55),$total_symm_vec,scales)

DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 88 and 2